#### imports

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

#### functions

In [2]:
def load_image(image_path, mask_path):
    """load the image and its mask"""
    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    return image, mask

def detect_blue_white(image, mask):
    """detect blue-white veil areas in the image using the mask"""

    # crop the image using the mask so that we only work with the lesion
    cropped_image = cv2.bitwise_and(image, image, mask=mask)
    
    # convert the cropped image to HSV 
    hsv = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2HSV)
    
    # define bounds for color detection - HUE | SATURATION | VALUE (brightness)
    lower_blue = np.array([90, 20, 80])
    upper_blue = np.array([150, 150, 150])
    
    # creates a new mask where all pixels fall within the defined bounds
    detected_mask = cv2.inRange(hsv, lower_blue, upper_blue)
    
    return detected_mask

def coverage_ratio(detected_mask, mask):
    veil_area = np.sum(detected_mask > 0) # total area of the blue-white veil
    total_area = np.sum(mask > 0)
    
    coverage_ratio = round(veil_area / total_area, 4)
    
    return coverage_ratio


def blue_white_veil(image_path, mask_path):
    """calculate the coverage ratio of blue-white veil areas"""
    image, mask = load_image(image_path, mask_path)
    detected_mask = detect_blue_white(image, mask)
    result = coverage_ratio(detected_mask, mask)
    
    return result

#### display images

In [3]:
def test_images(image_path, mask_path, save_folder):
    '''used to test images to visually assess  the correctness of our algorithm'''
    image, mask = load_image(image_path, mask_path)
    detected_mask = detect_blue_white(image, mask)
    result = coverage_ratio(detected_mask, mask)
    
    # check if the save folder exists, if not, create it
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    
    """display the original, mask, and detected mask images with coverage ratio and saves them in given folder"""
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(mask, cmap='gray')
    plt.title('Manual Mask')
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(cv2.cvtColor(detected_mask, cv2.COLOR_GRAY2RGB))
    plt.title(f'Detected Mask')
    plt.axis('off')

    plt.suptitle(f'Coverage Ratio: {result:.2%}')
    
    # save the figure to the designated save folder
    save_path = os.path.join(save_folder, f"result_{os.path.basename(image_path)}.png")
    plt.savefig(save_path)
    plt.close()
    
    plt.show()
    

#### looping over images

In [4]:
image_paths = [
    '../good_bad_images/MEL/good/images/lucie/PAT_340_714_314.png',
    '../testing_features/testing_images/blue_white_veil/images/1.png',
    '../testing_features/testing_images/blue_white_veil/images/2.png',
    '../testing_features/testing_images/blue_white_veil/images/3.png',
    '../testing_features/testing_images/blue_white_veil/images/4.png',
    '../testing_features/testing_images/blue_white_veil/images/5.png',
    '../testing_features/testing_images/blue_white_veil/images/6.png',
    '../good_bad_images/MEL/good/images/lucie/PAT_995_1867_165.png',
    '../good_bad_images/MEL/good/images/lucie/PAT_995_1867_165.png',
    '../good_bad_images/MEL/bad/images/lucie/PAT_262_402_14.png',
    '../good_bad_images/MEL/good/images/lucie/PAT_995_1867_165.png',
    '../good_bad_images/MEL/good/images/lucie/PAT_995_1867_165.png',
    '../good_bad_images/NEV/good/images/PAT_672_1272_705.png',
    '../good_bad_images/MEL/good/images/lucie/PAT_995_1867_165.png',
    '../good_bad_images/MEL/good/images/lucie/PAT_995_1867_165.png',
    '../good_bad_images/SCC/bad/images/david/PAT_131_197_974.png',
    '../good_bad_images/MEL/good/images/lucie/PAT_995_1867_165.png',
    '../good_bad_images/SEK/good/images/PAT_893_1697_613.png',
    '../good_bad_images/MEL/good/images/lucie/PAT_995_1867_165.png',
    
]

mask_paths = [
    '../good_bad_images/MEL/good/masks/lucie/PAT_340_714_314_mask.png',
    '../testing_features/testing_images/blue_white_veil/masks/1.png',
    '../testing_features/testing_images/blue_white_veil/masks/2.png',
    '../testing_features/testing_images/blue_white_veil/masks/3.png',
    '../testing_features/testing_images/blue_white_veil/masks/4.png',
    '../testing_features/testing_images/blue_white_veil/masks/5.png',
    '../testing_features/testing_images/blue_white_veil/masks/6.png',
    '../good_bad_images/MEL/good/masks/lucie/PAT_995_1867_165_mask.png',
    '../good_bad_images/MEL/good/masks/lucie/PAT_995_1867_165_mask.png',
    '../good_bad_images/MEL/bad/masks/lucie/PAT_262_402_14_mask.png',
    '../good_bad_images/MEL/good/masks/lucie/PAT_995_1867_165_mask.png',
    '../good_bad_images/MEL/good/masks/lucie/PAT_995_1867_165_mask.png',
    '../good_bad_images/NEV/good/masks/PAT_672_1272_705_mask.png',
    '../good_bad_images/MEL/good/masks/lucie/PAT_995_1867_165_mask.png',
    '../good_bad_images/MEL/good/masks/lucie/PAT_995_1867_165_mask.png',
    '../good_bad_images/SCC/bad/masks/david/PAT_131_197_974_mask.png',
    '../good_bad_images/MEL/good/masks/lucie/PAT_995_1867_165_mask.png',
    '../good_bad_images/SEK/good/masks/PAT_893_1697_613_mask.png',
    '../good_bad_images/MEL/good/masks/lucie/PAT_995_1867_165_mask.png',
]

# Loop over the paths and test each image
for img_path, msk_path in zip(image_paths, mask_paths):
    test_images(img_path, msk_path, '../testing_features/testing_images/blue_white_veil/results')